In [1]:
!pip install moviepy pydub


  Using cached moviepy-2.1.2-py3-none-any.whl.metadata (6.9 kB)
  Using cached imageio-2.37.0-py3-none-any.whl.metadata (5.2 kB)
  Using cached imageio_ffmpeg-0.6.0-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached proglog-0.1.11-py3-none-any.whl.metadata (794 bytes)
  Using cached python_dotenv-1.1.0-py3-none-any.whl.metadata (24 kB)
  Using cached pillow-10.4.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (9.2 kB)
Using cached moviepy-2.1.2-py3-none-any.whl (126 kB)
Using cached imageio-2.37.0-py3-none-any.whl (315 kB)
Using cached imageio_ffmpeg-0.6.0-py3-none-manylinux2014_x86_64.whl (29.5 MB)
Using cached pillow-10.4.0-cp312-cp312-manylinux_2_28_x86_64.whl (4.5 MB)
Using cached proglog-0.1.11-py3-none-any.whl (7.8 kB)
Using cached python_dotenv-1.1.0-py3-none-any.whl (20 kB)
  Attempting uninstall: pillow
    Found existing installation: pillow 11.2.1
    Uninstalling pillow-11.2.1:
      Successfully uninstalled pillow-11.2.1


In [5]:
!pip install tqdm

In [6]:
import subprocess
from pydub import AudioSegment, silence
import os
from tqdm import tqdm

# 1. Extrair o áudio usando ffmpeg
def extrair_audio_com_ffmpeg(video_path, audio_path):
    comando = [
        "ffmpeg",
        "-i", video_path,
        "-vn",              
        "-acodec", "pcm_s16le",  
        "-ar", "16000",     
        "-ac", "1",         
        audio_path
    ]
    subprocess.run(comando, check=True)

# 2. Dividir o áudio nos silêncios E registrar tempos
def dividir_audio(audio_path, output_audio_folder, video_path, output_image_folder, min_silence_len=300, silence_thresh=-40):
    audio = AudioSegment.from_wav(audio_path)

    os.makedirs(output_audio_folder, exist_ok=True)
    os.makedirs(output_image_folder, exist_ok=True)

    # Identifica os trechos de áudio entre silêncios
    silent_ranges = silence.detect_silence(
        audio,
        min_silence_len=min_silence_len,
        silence_thresh=silence_thresh
    )

    # Transformar a lista para começar e terminar onde há som
    boundaries = [0] + [item for sublist in silent_ranges for item in sublist] + [len(audio)]

    # Pega os intervalos de fala
    for i in tqdm(range(0, len(boundaries)-1, 2)):
        start = boundaries[i]
        end = boundaries[i+1]
        palavra = audio[start:end]

        # Salva o trecho de áudio
        palavra.export(f"{output_audio_folder}/palavra_{i//2 + 1}.wav", format="wav")

        # Calcula tempo em segundos
        tempo_medio_ms = (start + end) / 2
        tempo_medio_seg = tempo_medio_ms / 1000

        # Extrai a imagem do vídeo nesse tempo
        salvar_frame(video_path, tempo_medio_seg, f"{output_image_folder}/imagem_{i//2 + 1}.jpg")

# 3. Salvar um frame do vídeo em um tempo específico
def salvar_frame(video_path, tempo_segundos, output_imagem_path):
    comando = [
        "ffmpeg",
        "-ss", str(tempo_segundos),
        "-i", video_path,
        "-frames:v", "1",
        "-q:v", "2",  
        output_imagem_path
    ]
    subprocess.run(comando, check=True)

# 4. Exemplo de uso
video_path = "/media/guilherme/ssd_m2_data/py_new_projects/fast_english/extract_data_video/data/video/Screen_Recording_20250428_225640_FunEasyLearn.mp4"
audio_path = "data/audio/audio_extraido.wav"
output_audio_folder = "palavras_extraidas_audio"
output_image_folder = "palavras_extraidas_imagem"

extrair_audio_com_ffmpeg(video_path, audio_path)
dividir_audio(audio_path, output_audio_folder, video_path, output_image_folder)

print("Finalizado!")


ffmpeg version 6.1.1-3ubuntu5 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 13 (Ubuntu 13.2.0-23ubuntu3)
  configuration: --prefix=/usr --extra-version=3ubuntu5 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --disable-omx --enable-gnutls --enable-libaom --enable-libass --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libglslang --enable-libgme --enable-libgsm --enable-libharfbuzz --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-libzimg --ena

Finalizado!


In [2]:
from moviepy.editor import VideoFileClip
from pydub import AudioSegment, silence
import os

# 1. Extrair o áudio do vídeo
def extrair_audio(video_path, audio_path):
    video = VideoFileClip(video_path)
    video.audio.write_audiofile(audio_path)

# 2. Dividir o áudio com base nos silêncios
def dividir_audio(audio_path, output_folder, min_silence_len=300, silence_thresh=-40):
    audio = AudioSegment.from_wav(audio_path)

    # Detecta onde estão os silêncios
    partes = silence.split_on_silence(
        audio,
        min_silence_len=min_silence_len,  # Mínimo de ms para considerar como silêncio
        silence_thresh=silence_thresh     # Threshold de volume para considerar silêncio
    )

    # Salvar cada parte
    os.makedirs(output_folder, exist_ok=True)
    for i, parte in enumerate(partes):
        parte.export(f"{output_folder}/palavra_{i+1}.wav", format="wav")

# 3. Exemplo de uso
video_path = "/media/guilherme/ssd_m2_data/py_new_projects/fast_english/extract_data_video/data/video/Screen_Recording_20250428_225640_FunEasyLearn.mp4"
audio_path = "data/audio/audio_extraido.wav"
output_folder = "palavras_extraidas"

extrair_audio(video_path, audio_path)
dividir_audio(audio_path, output_folder)

print("Processo concluído!")


ModuleNotFoundError: No module named 'moviepy.editor'